# Next Smaller Element

In [ ]:
def nearest_smaller_to_right(arr):
    result = []
    stack = []
    
    for ele in arr[::-1]:
        if stack and ele <= stack[-1]:
            while stack and ele <= stack[-1]:
                stack.pop()
        
        result.append(stack[-1] if stack else -1)
        stack.append(ele)
        
    return result[::-1]

# LRU cache

In [ ]:
# Something Wrong
class Node:
    def __init__(self, key, val):
        self.key = key
        self.val = val
        self.next = None
        self.prev = None

class LRUCache:
    def __init__(self, capacity):
        self.head = Node(-1, -1)
        self.tail = Node(-1, -1)
        self.capacity = capacity
        self.map = dict()
        
        self.head.next = self.tail
        self.tail.prev = self.head
        
    def addNode(self, node):
        temp = self.head.next
        node.next = temp
        node.prev = self.head
        self.head.next = node
        temp.prev = node
        
    def deleteNode(self, node):
        delprev = node.prev
        delnext = node.next
        delprev.next = delnext
        delnext.prev = delprev
        
    def get(self, key):
        if key in self.map:
            resnode = self.map[key]
            res = resnode.val 
            del self.map[key]
            self.addNode(resnode)
            self.map[key] = self.head.next
            return res 
        
        return -1
 
    def put(self, key, value):
        if key in self.map:
            node = self.map[key]
            del self.map[key]
            self.deleteNode(node)
        
        if len(self.map) == self.capacity:
            if self.tail.prev.key in self.map:
                del self.map[self.tail.prev.key]
            self.map[key] = self.head.next
        
        self.addNode(Node(key, value))
        self.map[key] = self.head.next

# Working
from collections import OrderedDict

class LRUCache:
    def __init__(self, capacity):
        self.c = capacity
        self.d = OrderedDict()

    def get(self, key):
        if key in self.d:
            v = self.d[key]
            del self.d[key]
            self.d[key] = v
        return self.d.get(key,-1)

    def set(self, key, value):
        if key in self.d:
            v = self.d[key]
            del self.d[key]
            self.d[key] = v
        else:
            if len(self.d) == self.c:
                self.d.popitem(last=False)
        self.d[key] = value

# LFU Cache

In [4]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.cnt = 1
        self.next = None
        self.prev = None
        
class LinkedList:
    def __init__(self):
        self.size = 0
        self.head = Node(0, 0)
        self.tail = Node(0, 0)
        self.head.next = self.tail
        self.tail.prev = self.head
        
    def addFront(self, node):
        temp = self.head.next
        node.next = temp
        node.prev = self.head
        self.head.next = node
        temp.prev = node
        self.size += 1
    
    def removeNode(self, node):
        delprev = node.prev
        delnext = node.next
        delprev.next = delnext
        delnext.prev = delprev
        self.size -= 1

class LFUCache:

    def __init__(self, capacity: int):
        self.keyNode = dict()
        self.freqListMap = dict()
        
        self.maxSizeCache = capacity
        self.minFreq = 0
        self.curSize = 0

    def updateFreqListMap(self, node):
        del self.keyNode[node.key]
        self.freqListMap[node.cnt].removeNode(node)
        
        if node.cnt == self.minFreq and self.freqListMap[node.cnt].size == 0:
            self.minFreq += 1
        
        nextHigherFreqList = LinkedList()
        if node.cnt + 1 in self.freqListMap:
            nextHigherFreqList = self.freqListMap[node.cnt+1]
            
        node.cnt += 1
        nextHigherFreqList.addFront(node)
        self.freqListMap[node.cnt] = nextHigherFreqList
        self.keyNode[node.key] = node
        
    def get(self, key: int) -> int:
        if key in self.keyNode:
            node = self.keyNode[key]
            val = node.value
            self.updateFreqListMap(node)
            return val
        
        return -1

    def put(self, key: int, value: int) -> None:
        if self.maxSizeCache == 0:
            return
        
        if key in self.keyNode:
            node = self.keyNode[key]
            node.value = value
            self.updateFreqListMap(node)
        else:
            if self.curSize == self.maxSizeCache:
                li = self.freqListMap[self.minFreq]
                del self.keyNode[li.tail.prev.key]
                self.freqListMap[self.minFreq].removeNode(li.tail.prev)
                self.curSize -= 1
            self.curSize += 1
            self.minFreq = 1
            listFreq = LinkedList()
            
            if self.minFreq in self.freqListMap:
                listFreq = self.freqListMap[self.minFreq]
            
            node = Node(key, value)
            listFreq.addFront(node)
            self.keyNode[key] = node
            self.freqListMap[self.minFreq] = listFreq

# Largest rectangle in histogram

In [ ]:
class Solution:
    def NSL(self, A):
        stack = []
        result = []
        
        for i, ele in enumerate(A):
            while stack and stack[-1][0] >= ele:
                stack.pop()
            result.append((stack[-1][1]) if stack else -1)
            stack.append((ele, i))
            
        return result
    
    def NSR(self, A):
        n = len(A)
        stack = []
        result = []
        
        for i, ele in enumerate(A[::-1]):
            while stack and stack[-1][0] >= ele:
                stack.pop()
            result.append(stack[-1][1] if stack else n)
            stack.append((ele, n-i-1))
            
        return result[::-1]
    
    def largestRectangleArea(self, heights: List[int]) -> int:
        left = self.NSL(heights)
        right = self.NSR(heights)
        n = len(heights)
        area = [(heights[i] * (right[i]-left[i]-1)) for i in range(n)]
        
        return max(area)

# Sliding Window maximum

In [1]:
def slidingMaximum(arr, k):
    i = 0
    j = 0
    n = len(arr)

    result = []
    temp_arr = []

    while j < n:
        while temp_arr and temp_arr[-1] < arr[j]:
            temp_arr.pop()

        while temp_arr and temp_arr[0] < arr[j]:
                temp_arr.pop(0)
        temp_arr.append(arr[j])
        if j - i + 1 == k:
            result.append(temp_arr[0])
            if arr[i] == temp_arr[0]:
                temp_arr = temp_arr[1:]
            i += 1
        j += 1

    return result

# Implement Min Stack

In [ ]:
class MinStack:
    stack = []
    
    def push(self, x):
        if self.stack:
            minEle = min(x, self.stack[-1][1])
            self.stack.append((x, minEle))
        else:
            self.stack.append((x, x))

    # @return nothing
    def pop(self):
        if self.stack:
            self.stack.pop()
        # return -1

    # @return an integer
    def top(self):
        if self.stack:
            return self.stack[-1][0]
        return -1

    # @return an integer
    def getMin(self):
        if not self.stack:
            return -1
        return self.stack[-1][1]

# Rotten Orange (Using BFS)

In [ ]:
def orangesRotting(self, grid: List[List[int]]) -> int:
        count = 0
        visited = [[False for _ in range(len(grid[0]))] for _ in range(len(grid))]
        
        # Put all the indexes of rotten tomatoes in the queue
        queue = []
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 2:
                    queue.append((i, j, 0))
        
        # Do BFS
        while queue:
            curr_i, curr_j, curr_count = queue.pop(0)
            
            # If this node is already visited continue
            if visited[curr_i][curr_j]:
                continue
                
            visited[curr_i][curr_j] = True
            
            # Update the count 
            count = max(count, curr_count)
            
            for x, y in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                if 0 <= curr_i + x < len(grid) and 0 <= curr_j + y < len(grid[0]):
                    new_i = curr_i + x
                    new_j = curr_j + y
                    
                    if not visited[new_i][new_j] and grid[new_i][new_j] == 1:
                        queue.append((new_i, new_j, curr_count+1))
        
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 1 and visited[i][j] == False:
                    return -1
        
        return count

# Stock Span Problem

In [ ]:
def stock_span_problem(arr):
    result = []
    stack = []
    
    for i, ele in enumerate(arr):
        while stack and ele > stack[-1][0]:
            stack.pop()
                
        result.append(i - stack[-1][1] if stack else i + 1)
        stack.append((ele, i))
    
    return result

# Find maximum of minimums of every window size

In [ ]:
class Solution:
    def nextSmallestRight(self, arr):
        result = []
        stack = []
        n = len(arr)
        
        for i, ele in enumerate(arr[::-1]):
            while stack and stack[-1][0] >= ele:
                stack.pop()
            
            result.append(n if not stack else stack[-1][1])
            stack.append((ele, n-i-1))
        
        return result[::-1]
        
    def nextSmallestLeft(self, arr):
        result = []
        stack = []
        
        for i, ele in enumerate(arr):
            while stack and stack[-1][0] >= ele:
                stack.pop()
                
            result.append(-1 if not stack else stack[-1][1])
            stack.append((ele, i))
        
        return result
        
    def maxOfMin(self,arr,n):
        nsl = self.nextSmallestLeft(arr)
        nsr = self.nextSmallestRight(arr)
        
        ans = [0 for _ in range(n+1)]
        for i in range(n):
            l = nsr[i] - nsl[i] - 1
            ans[l] = max(arr[i], ans[l])
        
        for i in range(n-1, 0, -1):
            ans[i] = max(ans[i], ans[i+1])
        
        return ans[1:]

# The Celebrity Problem

In [ ]:
# Not Optimal
def celebrity(self, M, n):
        counter = {i: 0 for i in range(n)}
        people = [1 for _ in range(n)]
        
        for i in range(n):
            for j in range(n):
                if M[i][j] == 1:
                    counter[j] += 1
                    people[i] = 0
        
        for i in range(n):
            if people[i] == 1 and counter[i] == n-1:
                return i
        
        return -1

# Optimal: Using Two Pointers
def celebrity(self, M, n):
        i = 0
        j = n-1
        
        # If i knows j then i can't be a celebrity else vice-versa
        while i < j:
            if M[i][j] == 1:
                i += 1
            else:
                j -= 1
        
        # i is our potential celebrity now check if i is really a celebrity or not
        for j in range(n):
            if j == i:
                continue
            if M[i][j] == 1 or M[j][i] == 0:
                return -1
        
        return i